In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from sklearn import tree
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle 
pd.set_option('display.max_columns', None)

In [2]:
# load in our data
big_df = pd.read_csv("../data/big_df.csv")
big_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (215,216) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,lev1,blue_name,red_name,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
0,0,Glover Teixeira,Jan Blachowicz,7.088018,-6.329031,17.385159,9.229631,10.297141,15.558662,1979-10-28,1983-02-24,0.672043,0.824176,0,0,0.664179,0.729730,0.259259,0.402516,0.565517,0.732824,0.367347,0.274725,0.378099,0.441397,0.606941,0.628480,0.725490,0.748148,0.201

In [3]:
# isolating our future fights to predict
future_df = big_df[big_df["winner"] == "tbd"]
future_df

,lev1,blue_name,red_name,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
0,0,Glover Teixeira,Jan Blachowicz,7.088018,-6.329031,17.385159,9.229631,10.297141,15.558662,1979-10-28,1983-02-24,0.672043,0.824176,0,0,0.664179,0.729730,0.259259,0.402516,0.565517,0.732824,0.367347,0.274725,0.378099,0.441397,0.606941,0.628480,0.725490,0.748148,0.201

In [4]:
# Label encode our stances and weight class 
number = LabelEncoder()
# data['red_name'] = number.fit_transform(data['red_name'].astype('str'))
# data['blue_name'] = number.fit_transform(data['blue_name'].astype('str'))
future_df['red_stance'] = number.fit_transform(future_df['red_stance'].astype('str'))
future_df['blue_stance'] = number.fit_transform(future_df['blue_stance'].astype('str'))
future_df['weight_class'] = number.fit_transform(future_df['weight_class'].astype('str'))
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

,lev1,blue_name,red_name,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
0,0,Glover Teixeira,Jan Blachowicz,7.088018,-6.329031,17.385159,9.229631,10.297141,15.558662,1979-10-28,1983-02-24,0.672043,0.824176,0,0,0.664179,0.729730,0.259259,0.402516,0.565517,0.732824,0.367347,0.274725,0.378099,0.441397,0.606941,0.628480,0.725490,0.748148,0.201

In [5]:
# drop columns
data = future_df.drop(columns=["lev1", "winner", "red_name", "blue_name", "date", "event", "red_DOB", "blue_DOB", "method",\
                               "location", "red_nickname", "blue_nickname", "red_weight", "blue_weight", "index"])
# target = train_test_df["winner"]
# feature_names = data.columns

In [6]:
# load in our saved random forest model...
loaded_model = pickle.load(open("../saved_models/rf_up_model.sav", 'rb'))
pred = loaded_model.predict(data)
future_df["prediction"] = pred
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,lev1,blue_name,red_name,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index,prediction
0,0,Glover Teixeira,Jan Blachowicz,7.088018,-6.329031,17.385159,9.229631,10.297141,15.558662,1979-10-28,1983-02-24,0.672043,0.824176,0,0,0.664179,0.729730,0.259259,0.402516,0.565517,0.732824,0.367347,0.274725,0.378099,0.441397,0.606941,0.628480,0.725490,0.7

In [7]:
# Getting percentage certainty of prediction as an array of two outcomes, totaling 100
pcts = loaded_model.predict_proba(data)
# Looping through to get our favorite and dog in each fight
favs = []
dogs = []
for pct in pcts:
    fav = max(pct)
    dog = min(pct)
    favs.append(fav)
    dogs.append(dog)

In [8]:
# and adding these percentages to the dataframe
future_df["fav_pct"] = favs
future_df["dog_pct"] = dogs
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,lev1,blue_name,red_name,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index,prediction,fav_pct,dog_pct
0,0,Glover Teixeira,Jan Blachowicz,7.088018,-6.329031,17.385159,9.229631,10.297141,15.558662,1979-10-28,1983-02-24,0.672043,0.824176,0,0,0.664179,0.729730,0.259259,0.402516,0.565517,0.732824,0.367347,0.274725,0.378099,0.441397,0.606941,0.628

In [9]:
# round off percentages to make for cleaner viewing
future_df["fav_pct"] = future_df["fav_pct"].round(2)
future_df["dog_pct"] = future_df["dog_pct"].round(2)
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,lev1,blue_name,red_name,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index,prediction,fav_pct,dog_pct
0,0,Glover Teixeira,Jan Blachowicz,7.088018,-6.329031,17.385159,9.229631,10.297141,15.558662,1979-10-28,1983-02-24,0.672043,0.824176,0,0,0.664179,0.729730,0.259259,0.402516,0.565517,0.732824,0.367347,0.274725,0.378099,0.441397,0.606941,0.628

In [10]:
# Converting percentages to a moneyline as would be seen at a fight bookie, without the house weighting
future_df["fav_moneyline"] = (future_df["fav_pct"] * 100) / (100 - (future_df["fav_pct"] * 100)) * (-100)
future_df["dog_moneyline"] = ((100 - (future_df["dog_pct"] * 100)) / (future_df["dog_pct"] * 100) * 100)
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,lev1,blue_name,red_name,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,0,Glover Teixeira,Jan Blachowicz,7.088018,-6.329031,17.385159,9.229631,10.297141,15.558662,1979-10-28,1983-02-24,0.672043,0.824176,0,0,0.664179,0.729730,0.259259,0.402516,0.565517,0.732824,0.367347,0.274725,0.37

In [11]:
# convert moneylines to int
future_df["fav_moneyline"] = future_df["fav_moneyline"].astype(int)
future_df["dog_moneyline"] = future_df["dog_moneyline"].astype(int)
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,lev1,blue_name,red_name,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,0,Glover Teixeira,Jan Blachowicz,7.088018,-6.329031,17.385159,9.229631,10.297141,15.558662,1979-10-28,1983-02-24,0.672043,0.824176,0,0,0.664179,0.729730,0.259259,0.402516,0.565517,0.732824,0.367347,0.274725,0.37

In [12]:
# load in past picks as we will add our new picks to this running tally
my_picks = pd.read_csv("picks/rolling_picks.csv")
my_picks = my_picks.drop(columns="Unnamed: 0")
my_picks.head(20)

,blue_name,red_name,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,Ryan Spann,Anthony Smith,red,0.52,0.48,-108,108
1,Devin Clark,Ion Cutelaba,blue,0.56,0.44,-127,127
2,Cameron VanCamp,Nikolas Motta,red,0.54,0.46,-117,117
3,Christos Giagos,Arman Tsarukyan,blue,0.56,0.44,-127,127
4,Antonio Arroyo,Joaquin Buckley,red,0.57,0.43,-132,132
5,Tafon Nchukwi,Mike Rodriguez,red,0.55,0.45,-122,122
6,Raquel Pennington,Pannie Kianzad,red,0.70,0.30,-233,233
7,Tony Gravely,Nate Maness,red,0.61,0.39,-156,156
8,Rong Zhu,Dakota Bush,red,0.63,0.37,-170,170
9,JP Buys,Montel Jackson,red,0.73,0.27,-270,270


In [13]:
# dropping everything else to create a clean and simple dataframe of our future predictions 
future_picks = future_df[["blue_name", "red_name", "prediction", "fav_pct", "dog_pct", \
                          "fav_moneyline", "dog_moneyline"]]
future_picks

,blue_name,red_name,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,Glover Teixeira,Jan Blachowicz,red,0.66,0.34,-194,194
1,Cory Sandhagen,Petr Yan,red,0.67,0.33,-203,203
2,Dan Hooker,Islam Makhachev,red,0.65,0.35,-185,185
3,Marcin Tybura,Alexander Volkov,red,0.60,0.40,-150,150
4,Khamzat Chimaev,Li Jingliang,blue,0.58,0.42,-138,138
5,Volkan Oezdemir,Magomed Ankalaev,red,0.66,0.34,-194,194
6,Virna Jandiroba,Amanda Ribas,red,0.70,0.30,-233,233
7,Zubaira Tukhugov,Ricardo Ramos,blue,0.54,0.46,-117,117
8,Roman Kopylov,Albert Duraev,red,0.55,0.45,-122,122
9,Benoit Saint-Denis,Elizeu Zaleski dos Santos,red,0.56,0.44,-127,127


In [14]:
# This model is my prefered model so i have a running csv of all the predictions made by this model version
my_picks = pd.concat([future_picks, my_picks])
my_picks

,blue_name,red_name,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,Glover Teixeira,Jan Blachowicz,red,0.66,0.34,-194,194
1,Cory Sandhagen,Petr Yan,red,0.67,0.33,-203,203
2,Dan Hooker,Islam Makhachev,red,0.65,0.35,-185,185
3,Marcin Tybura,Alexander Volkov,red,0.60,0.40,-150,150
4,Khamzat Chimaev,Li Jingliang,blue,0.58,0.42,-138,138
...,...,...,...,...,...,...,...
67,Rafa Garcia,Chris Gruetzemacher,blue,0.60,0.40,-150,150
68,Kai Kamaka,Danny Chavez,blue,0.51,0.49,-104,104
69,Ashley Yoder,Jinh Yu Frey,red,0.58,0.42,-138,138
70,Zarrukh Adashev,Ryan Benoit,blue,0.60,0.40,-150,150


In [15]:
# my_picks.to_csv("picks/rolling_picks.csv")

In [16]:
# It is important to note that the moneyline is simply a straight calculation of percentage to moneyline, NOT how
# an actual moneyline would be presented at a sports book. Because bookies need to make money, and the moneylines
# they assign can be arbitrary, there will not be a straight 1-1 relationship between the favorite and dog. This is
# because bookies will account for outsides cases of draws and no contests where neither fighter is delared winner.
# It is also the case that bookies are required to make a profit, and will sometimes list BOTH fighters at negative
# money, meaning neither fighter will return more than a dollar for every dollar you bet on them to win. This is 
# seen in fights that are considered a coin flip where the "underdog" will have a moneyline of -110 (or something
# similar) and the "favorite" will be at -115 (or something similar)

# This predictive model SHOULD NOT be used for the purpose of betting without consideration of all these factors,
# and instead should be used as merely one addition datapoint to factor in if you are intending to bet on fights